In [1]:
%cd ../..
%pip install .
import time
# def placeObjs(car, numCars):
#     for i in range(numCars):
#         car = Car ahead of car by Range(4, 5)
#         leftCar = Car left of car by Normal(2, 0.1), facing roadDirection
#         rightCar = Car right of car by Normal(3, 0.1), facing Range(0, 10) deg relative to ego.heading
#     return leftCar, rightCar

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point, with visible_distance 200

# leftCar, rightCar = placeObjs(ego, 2)
# require (distance to leftCar) < 200
# require (distance to rightCar) < 200

/home/yongming/workspace/research/apperception_new_local/apperception
Processing /home/yongming/workspace/research/apperception_new_local/apperception
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for apperception: filename=apperception-0.1.0-py3-none-any.whl size=74298 sha256=fa8aefd07134bb35a7ebbf0eff537e0e353475776c38c1e8691bf9a08adfe29e
  Stored in directory: /tmp/pip-ephem-wheel-cache-t10pcxnr/wheels/a4/33/17/59e20ccef806baa1a68832d19de43f0fe8d640854903fd1772
Successfully built apperception
  Attempting uninstall: apperception
    Found existing installation: apperception 0.1.0
    Uninstalling apperception-0.1.0:
      Successfully uninstalled apperception-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  getX(t1.traj, cam.timestamp) < getX(cam.egoTranslation, cam.timestamp) AND
  getX(t2.traj, cam.timestamp) > getX(cam.egoTranslation, cam.timestamp) AND
  facingRelative(t1.heading, road_direction(t1.centroid, cam.timestamp), cam.timestamp) = 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) >= 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < 10 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 200 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 200
"""
"""
Need to implement:
  SQL function getX(tgeompoints, timestamptz)
  SQL function getX(geometry)
  SQL function road_direction(tgeompoint, timestamptz)
  SQL function road_direction(geometry)
  SQL function facingRelative(tfloat, real, timestamptz)
  SQL function facingRelative(real, real)
  SQL function distance(geometry, tgeompoint, timestamptz)
  SQL function distance(geometry, geometry)
"""

'\nNeed to implement:\n  SQL function getX(tgeompoints, timestamptz)\n  SQL function getX(geometry)\n  SQL function road_direction(tgeompoint, timestamptz)\n  SQL function road_direction(geometry)\n  SQL function facingRelative(tfloat, real, timestamptz)\n  SQL function facingRelative(real, real)\n  SQL function distance(geometry, tgeompoint, timestamptz)\n  SQL function distance(geometry, geometry)\n'

In [3]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

# ### Query ###
from apperception.utils import F
car_world = world.filter(lambda obj: obj.object_type == 'vehicle.car')

def pred_1(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp)
    )
world_1 = car_world.filter(pred_1)

In [5]:
start = time.time()

key_1 = world_1.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((getX(table_0.trajCentroids, Cameras.timestamp)<getX(Cameras.egoTranslation, Cameras.timestamp)) AND (getX(table_1.trajCentroids, Cameras.timestamp)>getX(Cameras.egoTranslation, Cameras.timestamp)))
        
done execute node
82.2858681678772


In [ ]:
def pred_2(obj1, obj2, cam):
    return (
#         F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0
        F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) < 10
    )
world_2 = car_world.filter(pred_2)

In [ ]:
start = time.time()

key_2 = world_2.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

In [ ]:
def pred_3(obj1, obj2, cam):
    return (
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
world_3 = car_world.filter(pred_3)

In [ ]:
start = time.time()

key_3 = world_3.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

In [ ]:
def pred(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp) and
#         F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) < 10 and
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
final_world = car_world.filter(pred)
start = time.time()
keys = final_world.get_id_and_time(num_joined_tables=2)
end = time.time()
print(format(end-start))

In [7]:
keys

[('bd26c2cdb22d4bb1834e808c89128898',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('a0049f95375044b8987fbcca8fda1e2b',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('86ed8530809d4b1b8fbc53808f599339',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('61dd7d03d7ad466d89f901ed64e2c0dd',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('56a71c208ac6472f90b6a82529a6ce61',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('42641eb6adcb4f8f8def8ef129d9e843',
  'c1958768d48640948f6053d04cffd35b',
  datetime.datetime(2018, 7, 23, 20, 29, 0, 697423, tzinfo=datetime.timezone.utc)),
 ('4080c30aa7104d91ad005a50b18f6108',
  